In [4]:
import os
import sys
os.environ['SPARK_HOME']='/home/samar/spark'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.9-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.appName('SparkJDBC') \
.config('spark.warehouse.dir','/user/hive/warehouse') \
.config('spark.driver.extraClassPath', 
        '/usr/share/java/mysql-connector-java.jar') \
.config('spark.executor.extraClassPath', 
        '/usr/share/java/mysql-connector-java.jar') \
.enableHiveSupport().getOrCreate()

In [3]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'3.0.1'

In [4]:
sc = spark.sparkContext

In [14]:
prop = {'user': 'root', 'password': 'puneetRai10198)', 'driver': 'com.mysql.jdbc.Driver'}
url = 'jdbc:mysql://localhost:3306/testdb'
mockdf = spark.read.format('jdbc').option('url', url).option('dbtable', 'mocktbl') \
.option('user', 'root').option('password', 'puneetRai10198)') \
.option('driver', 'com.mysql.jdbc.Driver').load()
mockdf.show()

+---+---------+----------+--------------------+----------+---------------+
| id|    fname|     lname|               email|    gender|         ipaddr|
+---+---------+----------+--------------------+----------+---------------+
|  1|  Adelina|   Nordass|  anordass0@cnbc.com|  Bigender|  240.102.55.70|
|  2|    Lisha|     oldey|     loldey1@ovh.net|Non-binary|  89.176.188.43|
|  3|  Geordie|    Sheere|    gsheere2@ask.com|Genderquee|224.104.196.249|
|  4|  Giacobo| Gostridge|   ggostridge3@de.vu|Genderflui| 107.166.210.39|
|  5| Nannette|  Shawyers|  nshawyers4@msn.com|      Male| 171.151.79.205|
|  6|    Colan|   Sharman|csharman5@purevol...|Genderflui|   182.59.94.45|
|  7|  Bellina|   Slatter|  bslatter6@admin.ch|  Bigender|  42.116.182.12|
|  8| Catarina|  Greening|cgreening7@odnokl...|Genderflui|  30.85.129.238|
|  9|Morganica|    Crowne|     mcrowne8@ca.gov|    Female|113.125.199.117|
| 10|    Ruddy|   Borland|rborland9@howstuf...|Non-binary|  10.32.109.107|
| 11|    Wiley|  Blaylock

In [15]:
mockdf.printSchema()

root
 |-- id: integer (nullable = true)
 |-- fname: string (nullable = true)
 |-- lname: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ipaddr: string (nullable = true)



In [16]:
mockdf.rdd.getNumPartitions()

1

In [17]:
mockdf.groupBy('lname').count().show()

+----------+-----+
|     lname|count|
+----------+-----+
|  Ganiford|    1|
|   Beecker|    1|
|   Naisbet|    1|
| Kempshall|    1|
|  Hanscome|    1|
|   McMakin|    1|
|     Guyon|    1|
|     Mairs|    1|
|    Laight|    1|
|Plumbridge|    1|
|Pitcaithly|    1|
|      Chad|    1|
|    Tester|    1|
|     Sager|    1|
|   Bunston|    1|
|    Canero|    1|
|     Lghan|    1|
|    Denial|    1|
|Martinetto|    1|
|     Riche|    1|
+----------+-----+
only showing top 20 rows



In [18]:
mockdf_part = spark.read.format('jdbc').option('url', url).option('dbtable', 'mocktbl') \
.option('user', 'root').option('password', 'puneetRai10198)') \
.option('driver', 'com.mysql.jdbc.Driver') \
.option('partitionColumn', 'id').option('lowerBound', 0).option('upperBound', 1000) \
.option('numPartitions', 4).load()
mockdf_part.show()

+---+---------+----------+--------------------+----------+---------------+
| id|    fname|     lname|               email|    gender|         ipaddr|
+---+---------+----------+--------------------+----------+---------------+
|  1|  Adelina|   Nordass|  anordass0@cnbc.com|  Bigender|  240.102.55.70|
|  2|    Lisha|     oldey|     loldey1@ovh.net|Non-binary|  89.176.188.43|
|  3|  Geordie|    Sheere|    gsheere2@ask.com|Genderquee|224.104.196.249|
|  4|  Giacobo| Gostridge|   ggostridge3@de.vu|Genderflui| 107.166.210.39|
|  5| Nannette|  Shawyers|  nshawyers4@msn.com|      Male| 171.151.79.205|
|  6|    Colan|   Sharman|csharman5@purevol...|Genderflui|   182.59.94.45|
|  7|  Bellina|   Slatter|  bslatter6@admin.ch|  Bigender|  42.116.182.12|
|  8| Catarina|  Greening|cgreening7@odnokl...|Genderflui|  30.85.129.238|
|  9|Morganica|    Crowne|     mcrowne8@ca.gov|    Female|113.125.199.117|
| 10|    Ruddy|   Borland|rborland9@howstuf...|Non-binary|  10.32.109.107|
| 11|    Wiley|  Blaylock

In [19]:
mockdf_part.rdd.getNumPartitions()

4

In [20]:
filter_query = "(select fname, lname from mocktbl where id between 10 and 20) fq"
push_down_df = spark.read.jdbc(url=url, table=filter_query, properties=prop)
push_down_df.show()

+--------+----------+
|   fname|     lname|
+--------+----------+
|   Ruddy|   Borland|
|   Wiley|  Blaylock|
|L;urette|Rosenkranc|
|    Izak|Hackworthy|
|     Des|  Prandini|
|  Nikola|    Treeby|
| Gregory|    Rebert|
|    Ravi|   Mc Harg|
|  Miller|  Shadrach|
|    Angy|  McJerrow|
|    Ryon|   Erswell|
+--------+----------+



In [21]:
push_down_df.explain()

== Physical Plan ==
*(1) Scan JDBCRelation((select fname, lname from mocktbl where id between 10 and 20) fq) [numPartitions=1] [fname#190,lname#191] PushedFilters: [], ReadSchema: struct<fname:string,lname:string>




In [22]:
df_pushdown = spark.read.jdbc(table="mocktbl", url=url, properties=prop).where('id between 10 and 20')

In [23]:
df_pushdown.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(mocktbl) [numPartitions=1] [id#203,fname#204,lname#205,email#206,gender#207,ipaddr#208] PushedFilters: [*IsNotNull(id), *GreaterThanOrEqual(id,10), *LessThanOrEqual(id,20)], ReadSchema: struct<id:int,fname:string,lname:string,email:string,gender:string,ipaddr:string>




In [24]:
df_pushdown.show()

+---+--------+----------+--------------------+----------+--------------+
| id|   fname|     lname|               email|    gender|        ipaddr|
+---+--------+----------+--------------------+----------+--------------+
| 10|   Ruddy|   Borland|rborland9@howstuf...|Non-binary| 10.32.109.107|
| 11|   Wiley|  Blaylock| wblaylocka@sohu.com|    Female| 127.107.56.54|
| 12|L;urette|Rosenkranc|lrosenkrancb@netw...|      Male| 210.212.50.19|
| 13|    Izak|Hackworthy|ihackworthyc@stor...|Genderquee|18.240.245.239|
| 14|     Des|  Prandini|dprandinid@typepa...|Genderquee| 95.22.157.244|
| 15|  Nikola|    Treeby|    ntreebye@ihg.com|Genderflui| 154.149.60.69|
| 16| Gregory|    Rebert|  grebertf@cam.ac.uk|   Agender|  77.41.90.191|
| 17|    Ravi|   Mc Harg|   rmchargg@wisc.edu|  Bigender|  239.38.73.82|
| 18|  Miller|  Shadrach| mshadrachh@admin.ch|Genderquee| 90.131.244.21|
| 19|    Angy|  McJerrow|amcjerrowi@blinkl...|    Female| 226.64.21.176|
| 20|    Ryon|   Erswell|  rerswellj@cnet.com|Gende

In [25]:
mockdf.write.jdbc(url=url, mode='append', table='mocktblcp', properties=prop)

In [26]:
spark.read.format('jdbc').option('url', url).option('dbtable', 'mocktblcp') \
.option('user', 'root').option('password', 'puneetRai10198)') \
.option('driver', 'com.mysql.jdbc.Driver').load().count()

1000